In [ ]:
%load_ext autoreload
%autoreload 2

# POS and NER tagging of dat

In [1]:

# Project Gutenberg, 244: A Study in Scarlet (en), Arthur Conan Doyle
text_en = """
This was a lofty chamber, lined and littered with countless bottles.
Broad, low tables were scattered about, which bristled with retorts,
test-tubes, and little Bunsen lamps, with their blue flickering flames.
There was only one student in the room, who was bending over a distant
table absorbed in his work. At the sound of our steps he glanced round
and sprang to his feet with a cry of pleasure. “I’ve found it! I’ve
found it,” he shouted to my companion, running towards us with a
test-tube in his hand. “I have found a re-agent which is precipitated
by hæmoglobin, and by nothing else.” Had he discovered a gold mine,
greater delight could not have shone upon his features.
""".strip()

In [17]:
import nltk
import spacy

spacy.prefer_gpu()
nlp = spacy.load('en_core_web_sm')
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\roger\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\roger\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [21]:
tokens = nlp(text_en)
[token.text for token in tokens]

['This',
 'was',
 'a',
 'lofty',
 'chamber',
 ',',
 'lined',
 'and',
 'littered',
 'with',
 'countless',
 'bottles',
 '.',
 '\n',
 'Broad',
 ',',
 'low',
 'tables',
 'were',
 'scattered',
 'about',
 ',',
 'which',
 'bristled',
 'with',
 'retorts',
 ',',
 '\n',
 'test',
 '-',
 'tubes',
 ',',
 'and',
 'little',
 'Bunsen',
 'lamps',
 ',',
 'with',
 'their',
 'blue',
 'flickering',
 'flames',
 '.',
 '\n',
 'There',
 'was',
 'only',
 'one',
 'student',
 'in',
 'the',
 'room',
 ',',
 'who',
 'was',
 'bending',
 'over',
 'a',
 'distant',
 '\n',
 'table',
 'absorbed',
 'in',
 'his',
 'work',
 '.',
 'At',
 'the',
 'sound',
 'of',
 'our',
 'steps',
 'he',
 'glanced',
 'round',
 '\n',
 'and',
 'sprang',
 'to',
 'his',
 'feet',
 'with',
 'a',
 'cry',
 'of',
 'pleasure',
 '.',
 '“',
 'I',
 '’ve',
 'found',
 'it',
 '!',
 'I',
 '’ve',
 '\n',
 'found',
 'it',
 ',',
 '”',
 'he',
 'shouted',
 'to',
 'my',
 'companion',
 ',',
 'running',
 'towards',
 'us',
 'with',
 'a',
 '\n',
 'test',
 '-',
 'tube',
 '

In [25]:
def add_tokens(lookup: dict, name: str, tagged_tokens: list[tuple[str,str]]):
  for token, tag in tagged_tokens:
    if not token.isalpha():
      continue
    if not token in lookup:
      lookup[token] = {'token': token}
    lookup[token][name] = tag

def get_pos_nltk(text: str, tagset: str=None):
  tokens = nltk.word_tokenize(text_en)
  return nltk.pos_tag(tokens, tagset=tagset)

def get_pos_spacy(text: str):
  tokens = nlp(text)
  return [(token.text, token.pos_) for token in tokens]


lookup = {}
add_tokens(lookup, 'nltk(standard)', get_pos_nltk(text_en))
add_tokens(lookup, 'nltk(universal)', get_pos_nltk(text_en, 'universal'))
add_tokens(lookup, 'spaCy', get_pos_spacy(text_en))
# list(lookup.values())


In [26]:
headers = ['term', 'nltk (pos)', 'nltk (universal)', 'spaCy', 'LLM']
rows = []

for e in lookup.values():
  rows.append([e['token'], e['nltk(standard)'],e['nltk(universal)'],e['spaCy']])

Markdown(tabulate(rows, headers))



# for token in nltk.word_tokenize(text_en):
#   if not token.isalpha():
#     continue
#   row = [token]
#   row.append(nltk.pos_tag([token]))
#   row.append(nltk.pos_tag([token], tagset='universal'))
#   rows.append(row)
#   if len(rows) > 30:
#     break



KeyError: 'spaCy'